<a href="https://colab.research.google.com/github/sathwikmethari/diffusers_ex_collab/blob/main/lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from diffusers import DiffusionPipeline
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
pipe=DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0",torch_dtype=torch.float16).to(device)

**load_lora_weights**

( pretrained_model_name_or_path_or_dict: typing.Union[str, typing.Dict[str, torch.Tensor]], adapter_name: typing.Optional[str] = None**kwargs )

Parameters::

*pretrained_model_name_or_path_or_dict (str or os.PathLike or dict)* — See lora_state_dict().

*adapter_name (str, optional)* — Adapter name to be used for referencing the loaded adapter model. If not specified, it will use default_{i} where i is the total number of adapters being loaded.

*low_cpu_mem_usage (bool, optional)* — Speed up model loading by only loading the pretrained LoRA weights and not initializing the random weights.

*kwargs (dict, optional)* — See lora_state_dict().

Load LoRA weights specified in pretrained_model_name_or_path_or_dict into self.unet and self.text_encoder.

All kwargs are forwarded to self.lora_state_dict.
See lora_state_dict() for more details on how the state dict is loaded.
See load_lora_into_unet() for more details on how the state dict is loaded into self.unet.
See load_lora_into_text_encoder() for more details on how the state dict is loaded into self.text_encoder.

In [ ]:
pipe.load_lora_weights("CiroN2022/toy-face", weight_name="toy_face_sdxl.safetensors", adapter_name="toy")

In [ ]:
prompt = "toy_face of a hacker with a hoodie"

lora_scale = 0.9
image = pipe(
    prompt, num_inference_steps=30, cross_attention_kwargs={"scale": lora_scale}, generator=torch.manual_seed(0)
).images[0]
image

You can use multiple loras using *set_adapters()*

**Parameters**

*adapter_names (List[str] or str)* — The names of the adapters to use.

*adapter_weights (Union[List[float], float], optional)* — The adapter(s) weights to use with the UNet. If None, the weights are set to 1.0 for all the adapters.

can disable it using *disable_lora()*


In [ ]:
# pipe.load_lora_weights("nerijs/pixel-art-xl", weight_name="pixel-art-xl.safetensors", adapter_name="pixel")
# pipe.set_adapters(["toy", "pixel"], adapter_weights=[0.5, 0.3])

prompt = "toy_face of a pixelated hacker with a hoodie"

lora_scale = 1
image = pipe(
    prompt, num_inference_steps=30, cross_attention_kwargs={"scale": lora_scale}, generator=torch.manual_seed(0)
).images[0]
image

**Customize adapters strength**

For even more customization, you can control how strongly the adapter affects each part of the pipeline. For this, pass a dictionary with the control strengths (called “scales”) to set_adapters().

For example, here's how you can turn on the adapter for the down parts, but turn it off for the mid and up parts:

In [ ]:
pipe.enable_lora()  # enable lora again, after we disabled it above
prompt = "toy_face of a hacker with a hoodie, pixel art"
adapter_weight_scales = { "unet": { "down": 1, "mid": 0, "up": 0} }
pipe.set_adapters("pixel", adapter_weight_scales)
image = pipe(prompt, num_inference_steps=30, generator=torch.manual_seed(0)).images[0]
image

In [ ]:
pipe.config

**Manage adapters**

You have attached multiple adapters in this tutorial, and if you’re feeling a bit lost on what adapters have been attached to the pipeline’s components, use the *get_active_adapters()* method to check the list of active adapters:

In [ ]:
active_adapters = pipe.get_active_adapters()
active_adapters

In [ ]:
#You can also get the active adapters of each pipeline component with get_list_adapters():
list_adapters_component_wise = pipe.get_list_adapters()
list_adapters_component_wise